# Regression Benchmark 

### Probem Example (Regression) - Big Mart Sales:
To build a predictive model and find out the sales of each product at the store.

Good starting point:
- Mean - whats has been total sales of each product month on month
- Mean with respect to another variable

The two most commonly used measures of central tendency for numerical data are the mean and the median. Since the regression problem deals with continuous data, mean and median are the correct measures.


To evaluate the model:

Mean Absolute Error -> sum of abs different between every observation, divided by number of obs


In [1]:
#importing libraries 

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('train_bm.csv')

In [4]:
data.shape

(8523, 12)

In [56]:
data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [6]:
data.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

### Shuffling and Creating Train and Test Set

In [8]:
from sklearn.utils import shuffle

In [12]:
#shuffle dataset

data = shuffle(data, random_state=42)

#creating 4 division of data
div = int(data.shape[0]/4)

# 3 parts to train set and 1 part to test set
train = data.iloc[:3*div+1,:]
test = data.iloc[3*div+1:]

In [13]:
train.shape, test.shape, data.shape

((6391, 12), (2132, 12), (8523, 12))

## simple mean model (benchmark)

Find out the simple mean of train model and save it as a column in test dataset


In [15]:
test['simple_mean'] = train.Item_Outlet_Sales.mean()

<ipython-input-15-e4d80bf28089>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['simple_mean'] = train.Item_Outlet_Sales.mean()


Calculate the error between the mean values created in line above with the actual value in Item_Outlet_Sales()


In [57]:
from sklearn.metrics import mean_absolute_error as MAE

simple_mean_error = MAE(test.Item_Outlet_Sales, test.simple_mean)
simple_mean_error

1348.3091635746123

This is the benchmark value for model we will create to predict accuracy of the models.

## Mean Item Outlet Sales with respect to Outlet_Type

Now, we will try to improve the above prediction by predicting based on outlet tyoe


In [27]:
out_type = pd.pivot_table(train, values = 'Item_Outlet_Sales', index=['Outlet_Type'], aggfunc=np.mean)

out_type

,Item_Outlet_Sales
Outlet_Type,
Grocery Store,334.106148
Supermarket Type1,2293.636762
Supermarket Type2,2034.330733
Supermarket Type3,3684.008727


In [29]:
# initializing new column to zero
test['Out_type_mean'] = 0

for i in test.Outlet_Type.unique():
    test['Out_type_mean'][test.Outlet_Type == str(i)] = train['Item_Outlet_Sales'][train.Outlet_Type == str(i)].mean()

<ipython-input-29-9f7ddd1e0e07>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Out_type_mean'] = 0
<ipython-input-29-9f7ddd1e0e07>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Out_type_mean'][test.Outlet_Type == str(i)] = train['Item_Outlet_Sales'][train.Outlet_Type == str(i)].mean()
c:\Users\Neelam\Desktop\Desktop\Python\Video\env\lib\site-packages\pandas\core\series.py:963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [30]:
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,simple_mean,Out_type_mean
432,FDF10,15.50,Regular,0.157172,Snack Foods,149.1418,OUT049,1999,Medium,Tier 1,Supermarket Type1,588.5672,2169.533,2293.636762
4451,FDZ37,NaN,Regular,0.019673,Canned,86.4198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1918.8356,2169.533,3684.008727
1412,DRF23,4.61,Low Fat,0.123346,Hard Drinks,172.5396,OUT017,2007,NaN,Tier 2,Supermarket Type1,3663.2316,2169.533,2293.636762
1329,NCQ41,NaN,Low Fat,0.019386,Health and Hygiene,194.5794,OUT027,1985,Medium,Tier 3,Supermarket Type3,3511.4292,2169.533,3684.008727
6874,NCM17,7.93,Low Fat,0.071426,Health and Hygiene,45.9086,OUT018,2009,Medium,Tier 3,Supermarket Type2,1070.6064,2169.533,2034.330733


In [32]:
#mean suqare error

err = MAE(test.Item_Outlet_Sales, test.Out_type_mean)
err

1114.8889656414237

Conclusion: As we can see, the MAE did improve by using Outlet Type.

## Mean Item Outlet Sales with respect to Outlet_Establishment_Year


In [35]:
establis_year = pd.pivot_table(data, values='Item_Outlet_Sales', index= ['Outlet_Establishment_Year'], aggfunc=np.mean)

establis_year

,Item_Outlet_Sales
Outlet_Establishment_Year,
1985,2483.677474
1987,2298.995256
1997,2277.844267
1998,339.351662
1999,2348.354635
2002,2192.384798
2004,2438.841866
2007,2340.675263
2009,1995.498739


In [42]:
test['establisment_year_mean'] = 0

for i in data.Outlet_Establishment_Year.unique():
    test['establisment_year_mean'][test.Outlet_Establishment_Year == str(i)] = train['Item_Outlet_Sales'][train['Outlet_Establishment_Year'] == str(i)].mean()

<ipython-input-42-afb03e879e38>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['establisment_year_mean'] = 0
<ipython-input-42-afb03e879e38>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['establisment_year_mean'][test.Outlet_Establishment_Year == str(i)] = train['Item_Outlet_Sales'][train['Outlet_Establishment_Year'] == str(i)].mean()
c:\Users\Neelam\Desktop\Desktop\Python\Video\env\lib\site-packages\pandas\core\series.py:963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the document

In [43]:
errorrr = MAE(test.Item_Outlet_Sales, test.establisment_year_mean)

errorrr

2216.5290828330203

Conclusion: As we can see, the MAE did not improve by using establishment year.

## Mean Item_Outlet_Sales with respect to both Outlet_Location_Type and Outlet_Establishment_Year

Now we will use 2 features to create our model. Making the model more complex

In [46]:
combo_mean = pd.pivot_table(train, values='Item_Outlet_Sales', index = ['Outlet_Location_Type', 'Outlet_Establishment_Year'], aggfunc=np.mean)

combo_mean

Item_Outlet_Sales
Outlet_Location_Type Outlet_Establishment_Year                   
Tier 1               1985                              332.709060
                     1997                             2249.438082
                     1999                             2368.598566
Tier 2               2002                             2105.096784
                     2004                             2435.711052
                     2007                             2350.448072
Tier 3               1985                             3684.008727
                     1987                             2254.352110
                     1998                              335.469243
                     2009                             2034.330733

In [47]:
# Initiating new empty column
test['Super_mean'] = 0

# Assigning variables to strings ( to shorten code length)
s2 = 'Outlet_Location_Type'
s1 = 'Outlet_Establishment_Year'

# For every Unique Value in s1
for i in test[s1].unique():
  # For every Unique Value in s2
  for j in test[s2].unique():
    # Calculate and Assign mean to new column, corresponding to both unique values of s1 and s2 simultaneously
    test['Super_mean'][(test[s1] == i) & (test[s2]==str(j))] = train['Item_Outlet_Sales'][(train[s1] == i) & (train[s2]==str(j))].mean()

here(~key, value, inplace=True)
<ipython-input-47-1d3bf93b1d68>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Super_mean'][(test[s1] == i) & (test[s2]==str(j))] = train['Item_Outlet_Sales'][(train[s1] == i) & (train[s2]==str(j))].mean()
c:\Users\Neelam\Desktop\Desktop\Python\Video\env\lib\site-packages\pandas\core\series.py:963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)
<ipython-input-47-1d3bf93b1d68>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [48]:
#calculating mean absolute error
super_mean_error = MAE(test['Item_Outlet_Sales'] , test['Super_mean'] )
super_mean_error

1118.0230715619844

# Classification Benchmark

### Probem Example (Classification) - Titanic:
To predict wether a passenger of titanic would have survived or not?

Good point to start: Mode

To Evaluate: Accuracy - Correctly predict observation upon total observation


In [50]:
#importing libraries 
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score

In [54]:
data2 = pd.read_csv("train.csv")
data2.shape

(891, 12)

In [55]:
data2.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200
